# Moter dataset Anomaly detection

In [4]:
dataset = 'data/sensor_data_b.csv'

### Check dataset proportions

In [5]:
import pandas as pd 

df = pd.read_csv(dataset, header=None) 
label_counts = df.iloc[:, -1].value_counts()

print(label_counts)


3
0    15889
1     2056
2     2055
Name: count, dtype: int64


### dataset split 

In [9]:
label_0_data = df[df.iloc[:, -1] == 0]
label_1_data = df[df.iloc[:, -1] == 1]
label_2_data = df[df.iloc[:, -1] == 2]

label_0_data.to_csv('data/label_0_data.csv', index=False)
label_1_data.to_csv('data/label_1_data.csv', index=False)
label_1_data.to_csv('data/label_2_data.csv', index=False)

To detect anomalies, we plan to input data into the model at 1-second intervals with a sampling rate of 100Hz. Therefore, we will segment the data into 70 samples for each 1-second intervals.

In [7]:
import pandas as pd

label_0 = pd.read_csv("data/label_0_data.csv")
label_1 = pd.read_csv("data/label_1b_data.csv")
label_2 = pd.read_csv("data/label_2_data.csv")
label_3 = pd.read_csv("data/label_3_data.csv")

In [14]:
len_0 = len(label_0)
len_1 = len(label_1)
len_2 = len(label_2)
len_3 = len(label_3)

label_X = [label_0, label_1, label_2, label_3]
rem = [len_0, len_1, len_2, len_3]

for i in range(4):
    r = rem[i] % 100
    label_X[i] = label_X[i].iloc[:rem[i] - r]

    print(len(label_X[i]))



359900
359800
359900
359900


In [19]:
import numpy as np

In [32]:
data_split_0 = np.array_split(label_X[0], len(label_X[0]) // 100)
data_split_1 = np.array_split(label_X[1], len(label_X[1]) // 100)
data_split_2 = np.array_split(label_X[2], len(label_X[2]) // 100)
data_split_3 = np.array_split(label_X[3], len(label_X[3]) // 100)


y_label_0 = np.zeros(len(data_split_0))
y_label_1 = np.ones(len(data_split_1))
y_label_2 = np.full(len(data_split_2), 2)
y_label_3 = np.full(len(data_split_3), 3)


### Split train, test set

train_ratio = 0.7

In [56]:
train_ratio = 0.7
test_ratio = 0.3

data_splits = [data_split_0, data_split_1, data_split_2, data_split_3]
y_data_splits = [y_label_0, y_label_1, y_label_2, y_label_3]

train_sizes = []
train_data = []
test_data = []

y_train_data = []
y_test_data = []


for i in range(4):
    data_len = len(data_splits[i])
    train_size = int(data_len * train_ratio)
    train_sizes.append(train_size)
    train_data.append(data_splits[i][:train_size])
    test_data.append(data_splits[i][train_size:])

    y_train_data.append(y_data_splits[i][:train_size])
    y_test_data.append(y_data_splits[i][train_size:])


# check
for i in range(4):
    print(len(train_data[i]) + len(test_data[i]))

x_train_concat = np.concatenate(train_data, axis = 0)
x_test_concat = np.concatenate(test_data, axis = 0)
y_train_concat = np.concatenate(y_train_data, axis=0)
y_test_concat = np.concatenate(y_test_data, axis=0)

# check 
print(x_train_concat.shape[0] + x_test_concat.shape[0])
print(y_train_concat.shape[0] + y_test_concat.shape[0])

3599
3598
3599
3599
14395
14395
(4320,)
